<a href="https://colab.research.google.com/github/Maplemx/Agently/blob/main/playground/using_tools_to_enhance_your_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Tools to Enhance Your Agent

## Demo Description

**Author:** Agently Team

**Prompt Language:** English

**Agent Components:** Tool

**Description:**

Using Tools is the new feature in Agently v3.1.

You can register your function as tools to agent instance or to global tool manager.

You can also use default tools plugins in [`/plugins/tool/` dir](https://github.com/Maplemx/Agently/tree/main/src/plugins/tool) or create more tool package plugins. [Go to `/plugins/tool/`](https://github.com/Maplemx/Agently/tree/main/src/plugins/tool) to see how simple it is to create your own tool package plugins.

Then you can let agent instance plan whether or not to use tools and in what order to use those tools.

You can also use `.must_call()` alias to tell agent instance to generate a dict type result to help you calling a tool.

This show case will demostrate how to do that.

使用工具是Agently v3.1版本的重大功能性升级。

你可以把自己编写的函数作为工具注册给Agent实例，或者注册到全局的工具管理器（tool manager）上。

你也可以使用框架在[`/plugins/tool/`文件夹](https://github.com/Maplemx/Agently/tree/main/src/plugins/tool)中内置的工具集（tool package）插件，或者根据这些插件文件的格式，创建自己的工具集插件。[访问`/plugins/tool/`文件夹](https://github.com/Maplemx/Agently/tree/main/src/plugins/tool)你就会发现创建自己的工具集插件是一件多么简单的事情。

然后，向Agent实例以`.register_tool()`注册工具，或是通过`.use_all_public_tools()`等指令告知Agent启用全部或者某些工具，你就可以让Agent实例自行决定是否在处理请求时使用工具了。

当然，你也可以通过`.must_call()`指令让Agent实例生成一个字典类型的结果，帮助你来调用某个指定的工具。

下面的案例将会逐一展示使用Agently框架v3.1版本是如何做到这些事情的。

## Step 1: Install Packages

In [ ]:
!pip install -U Agently
!pip install pytz

## Step 2: Demo Code

### DEMO 1: Browse web page using tool 'browse'

In [8]:
import Agently

# You can change other models
# Read https://github.com/Maplemx/Agently/blob/main/docs/guidebook/application_development_handbook.ipynb
# to explore how to switch models by simply changing some settings
agent_factory = (
    Agently.AgentFactory()
        .set_settings("model.OpenAI.auth.api_key", "")
)

agent = agent_factory.create_agent()

# You can also use .use_all_public_tools() to enable all tools for agent
# But be caution: agent may plan to use more tools than your expectation
result = (
    agent
        .use_public_tools(["browse"])
        .input("https://zhuanlan.zhihu.com/p/33953044 what does this page say? List the key points.")
        .start()
)
print(result)

The key points of the article are:
1. Essential items to pack for travel, including documents, money, phone and charger, and personal items.
2. Different packing techniques for clothing to maximize luggage space.
3. Additional considerations based on the destination, such as beach or mountain travel.
4. Tips for preparing luggage and packing efficiently.


#### Current public tool list:

- search_info 搜索信息
- search_news 搜索新闻
- search_definition 搜索名词定义
- browse 浏览网页
- calculate （使用Python代码）完成计算



### DEMO 2: Resigter a customize tool to agent

In [11]:
from datetime import datetime
import Agently

agent_factory = (
    Agently.AgentFactory()
        .set_settings("model.OpenAI.auth.api_key", "")
)

# Define a tool to get current date and time
def get_datetime():
    return datetime.now()

agent = agent_factory.create_agent()

# When you register tool to agent instance
# the agent instance will automatically plan to use the tool
result = (
    agent
        .register_tool(
            tool_name="now",
            desc="get current date and time",
            args={},
            func=get_datetime,
        )
        .input("what time is it now?")
        .start()
)
print(result)

The current time is 2:46 PM on January 3rd, 2024.


You can also register the tool to global tool manager like this:

In [13]:
from datetime import datetime
import Agently

agent_factory = (
    Agently.AgentFactory()
        .set_settings("model.OpenAI.auth.api_key", "")
)

def get_datetime():
    return datetime.now()

# Register tool to global tool manager
Agently.global_tool_manager.register(
    tool_name="now",
    desc="get current date and time",
    args={},
    func=get_datetime
)

agent = agent_factory.create_agent()

# Then use .use_public_tools() to state the tool name that you want to use
result = (
    agent
        .use_public_tools(["now"])
        .input("what time is it now?")
        .start()
)
print(result)

The current time is 14:52:57


### DEMO 3: Call a tool using .must_call()

In [18]:
import pytz
import Agently

agent_factory = (
    Agently.AgentFactory()
        .set_settings("model.OpenAI.auth.api_key", "")
)

# Let's slightly modify this funciton
def get_datetime(timezone):
    tz = pytz.timezone(timezone)
    return datetime.now().astimezone(tz)

# Now it required an argument
Agently.global_tool_manager.register(
    tool_name="now",
    desc="get current date and time",
    args={
        "timezone": (
            "String",
            "[*Required]timezone string use in pytz.timezone() in Python"
        )
    },
    func=get_datetime
)

agent = agent_factory.create_agent()

# Use .must_call() to generate calling dict result without enough information
result = (
    agent
        .must_call("now")
        .input("what time is it now?")
        .start()
)
print("Without enough information:\n", result)

# Try again with enough information
result = (
    agent
        .must_call("now")
        .input("what time is it now? I'm in Beijing, China.")
        .start()
)
print("\nWith enough information:\n", result)
if result["can_call"]:
    print(get_datetime(**result["args"]))

Without enough information:
 {'can_call': False, 'args': {'timezone': None}, 'question': "What timezone are you in? (e.g. 'America/New_York')"}

With enough information:
 {'can_call': True, 'args': {'timezone': 'Asia/Shanghai'}, 'question': ''}
2024-01-03 23:04:54.426656+08:00


---

[**_<font color = "red">Agent</font><font color = "blue">ly</font>_** Framework - Speed up your AI Agent Native application development](https://github.com/Maplemx/Agently)